In [49]:
import numpy as np
import pandas as pd

# Nygaard TDM

In [50]:
U_0 = 10
z_0 = 0.0001
D = 100
z_H = 100
C_T = 0.8
s_x = 10
s_y = 10
s_d = s_x*D
s_c = s_y*D
kappa = 0.4

In [51]:
def calculate_wind_speed(x_1, x_2, x_3, z_H, D, C_T, s_d, s_c):

    c_t = (np.pi * C_T * D**2) / (8 * s_d * s_c)

    nu_star = 28 * np.sqrt(c_t)
    
    beta =  nu_star / (1 + nu_star)
    
    z_0WF = z_H * (1 + D / (2 * z_H))**beta * np.exp(-kappa * (c_t + (kappa / np.log((z_H / z_0) * (1 - D / (2 * z_H))**beta))**2)**-0.5)

    x = np.linspace(x_1, x_3-1, int(x_3-x_1))

    h_1 = np.full_like(x, np.nan, dtype=float)
    h_2 = np.full_like(x, np.nan, dtype=float)

    alpha_def = 0.8

    #ratio = 1 + 13450 / x_3**1.28
    #alpha_rec = 0.8 * ratio
    #print('alpha_rec:', alpha_rec)
    alpha_rec = 0.8
    a = 1 #0.859

    h_1[x >= x_1] = z_H + a * ((x[x >= x_1] - x_1) / z_0WF)**alpha_def * z_0WF
    h_2[x >= x_2] =   0 + a * ((x[x >= x_2] - x_2) / z_0WF)**alpha_rec * z_0WF

    U = np.full_like(x, np.nan, dtype=float)
    U[0] = U_0

    idx1 = h_1 >= z_H
    U[idx1] = U_0 * (np.log(h_1[idx1] / z_0) * np.log(z_H / z_0WF)) / (np.log(h_1[idx1] / z_0WF) * np.log(z_H / z_0))

    idx2 = (x >= x_2) & (h_2 >= z_H)
    U[idx2] = U_0 * (np.log(h_1[idx2] / z_0) * np.log(h_2[idx2] / z_0WF)) / (np.log(h_1[idx2] / z_0WF) * np.log(h_2[idx2] / z_0))

    for i in range(1, len(U)):
        if U[i] >= U_0 * 0.9999:
            U[i] = U_0 * 0.9999
        elif U[i] < 0:
            U[i] = 0

    return x, U, c_t

# Alex's firkanter :)

In [52]:
d = 400 * D

l_farm = int(10000)
w_farm = int(10000 + (2*D/2))
h_farm = D

x_1 = -l_farm
x_2 = 0
x_3 = d

x, u_tdm, c_t = calculate_wind_speed(x_1, x_2, x_3, z_H, D, C_T, s_d, s_c)

A_farm = l_farm * w_farm
A_exit = h_farm * w_farm
A_0 = 2/3 * A_exit

c_t_ny = c_t * A_farm/A_exit

u_s = (U_0 - u_tdm[-1]) / U_0
A_x = A_0 * c_t_ny / (2*(u_s - u_s**2))

kx = (- h_farm/4 - w_farm/4 
      + (np.sqrt(h_farm**2 + 2*A_0 + w_farm**2 - 4*A_0 + 4*(A_0 * (c_t_ny / (2*(u_s - u_s**2))))))/4)/d

k = int(kx*d)

print(f'c_t:     {c_t:.5f}')
print(f'U_meas:  {u_tdm[l_farm+4*D]:.3f}')
print(f'u_s:     {u_s:.5f}\n')
print(f'A_farm:  {A_farm:.0f}')
print(f'A_exit:  {A_exit:.0f}')
print(f'A_0:     {A_0:.0f}')
print(f'A_x:     {A_x:.0f}')
print(f'kx:      {kx:.5f}  [m/m]')
print(f'k:       {k:.0f}      [m]')

c_t:     0.00314
U_meas:  8.318
u_s:     0.00914

A_farm:  101000000
A_exit:  1010000
A_0:     673333
A_x:     11673631
kx:      0.01212  [m/m]
k:       484      [m]


# Save to CSV

In [53]:
y_meas = np.linspace(0 - k, w_farm + k, w_farm+(2*k)+1)
U = np.full_like(y_meas, u_tdm[-1])

In [54]:
data = pd.DataFrame({
    "y_wake": y_meas / D,
    "U_norm": U / U_0,
    "kx": kx,
})
data.to_csv(f"../ArbitraryLayouts/ValidationData/{int(d / D)}D.csv", index=False)